# Testing various models by playing match against each other

In [1]:
# Imports
from utils_1.goenv import GoEnv,_format_state, create_env_copy
from utils_1.fnet import NeuralTrainer, AlphaNeural
import numpy as np
from tqdm.auto import tqdm

## Simple Policy Player

In [2]:
class AlphaGoPlayer():
    def __init__(self, player_color, path = "utils_1/nov8/net30.model", play_random=False, debug=True):
        # setup parameters
        self.player_color = player_color
        self.random_player = play_random
        
        # init environment
        if player_color==1:
            self.env = GoEnv(player_color="black", board_size=13)
        else:
            self.env = GoEnv(player_color="white", board_size=13)
        self.env.reset()
        self.debug=debug
        
        # load neural network
        self.model = NeuralTrainer(res_blocks=10, board_size=13)
        self.model.load_model(path)
        
    def get_action(self, cur_state, opponent_action):
        if opponent_action>=0:
            self.env.step(opponent_action)
            if self.debug:
                print('-------------------------------------')
                print('opponent_action: ' + str(opponent_action))
                print('-------------------------------------')
        
        legal_moves = self.env.get_legal_moves()
        if self.random_player:
                actions = np.nonzero(self.env.get_legal_moves())[0]
                action = np.random.choice(actions)
        else:
            policy, value = self.model.predict(self.env.get_history())
            policy = policy*legal_moves
            action = np.argmax(policy)
            
        self.env.step(action)
        
        if self.debug:
            print('-------------------------------------')
            print('player_action: ' + str(action))
            print('-------------------------------------')
            self.env.print_board()
        
        return action

# Playing Matches

In [3]:
def play_matches(a1_configs, a2_configs, num_matches=20,
                 player_colors={'black': 'a1', 'white': 'a2'}, tournament_no=1):
    # Play matches between a1 and a2
    print ('Starting Tournament #%d' % tournament_no)
    print (player_colors)
    
    wins = {
        'a1': 0,
        'a2': 0
    }

    for match_no in tqdm(range(num_matches)):
        state = GoEnv("black", board_size=13)
        state.reset()
        
        a1 = AlphaGoPlayer(**a1_configs)
        a2 = AlphaGoPlayer(**a2_configs)

        a1_action = -1
        a2_action = -1

        for i in range(400):
            if(i==399):
                print("Limit reached")

            a1_action = a1.get_action('null', a2_action)
            _, _, done = state.step(a1_action)
            if (done or state.isComplete()):
                break   

            a2_action = a2.get_action('null', a1_action)
            _, _, done = state.step(a2_action)
            if (done or state.isComplete()):
                break

        winner = -1 * state.get_winner()
        if winner == 1:
            # It's black's victory
            wins[player_colors['black']] += 1
        else:
            # It's white victory
            wins[player_colors['white']] += 1

        if ((match_no + 1) % 10 == 0):
            # Log after every 10 matches
            print (wins)

    print(wins)
    print ('-----------------------------------------------------------------------------')

In [4]:
# Match 1
player_colors = {'black': 'a1', 'white': 'a2'}
a1_configs = {'player_color': 1, 'path': "utils_nov8/nov8/net37.model", 'debug': False}
a2_configs = {'player_color': 2, 'path': "utils_nov8/nov8/net55.model", 'debug': False}
play_matches(a1_configs, a2_configs, player_colors=player_colors, tournament_no=1)


Starting Tournament #1
{'black': 'a1', 'white': 'a2'}


{'a1': 10, 'a2': 0}
{'a1': 20, 'a2': 0}

{'a1': 20, 'a2': 0}
-----------------------------------------------------------------------------


In [5]:
# Match 2
player_colors = {'black': 'a2', 'white': 'a1'}
a1_configs = {'player_color': 2, 'path': "utils_nov8/nov8/net37.model", 'debug': False}
a2_configs = {'player_color': 1, 'path': "utils_nov8/nov8/net55.model", 'debug': False}
play_matches(a1_configs, a2_configs, player_colors=player_colors, tournament_no=1)

Starting Tournament #1
{'black': 'a2', 'white': 'a1'}


{'a1': 10, 'a2': 0}
{'a1': 20, 'a2': 0}

{'a1': 20, 'a2': 0}
-----------------------------------------------------------------------------


In [6]:
# Match 1=3
player_colors = {'black': 'a1', 'white': 'a2'}
a1_configs = {'player_color': 1, 'path': "utils_nov8/nov8/net37.model", 'debug': False}
a2_configs = {'player_color': 2, 'path': "utils_1/nov9-correct/net11.model", 'debug': False}
play_matches(a1_configs, a2_configs, player_colors=player_colors, tournament_no=3)


Starting Tournament #3
{'black': 'a1', 'white': 'a2'}


{'a1': 10, 'a2': 0}
{'a1': 20, 'a2': 0}

{'a1': 20, 'a2': 0}
-----------------------------------------------------------------------------


In [7]:
# Match 4
player_colors = {'black': 'a2', 'white': 'a1'}
a1_configs = {'player_color': 2, 'path': "utils_nov8/nov8/net37.model", 'debug': False}
a2_configs = {'player_color': 1, 'path': "utils_1/nov9-correct/net11.model", 'debug': False}
play_matches(a1_configs, a2_configs, player_colors=player_colors, tournament_no=4)


Starting Tournament #4
{'black': 'a2', 'white': 'a1'}


{'a1': 10, 'a2': 0}
{'a1': 20, 'a2': 0}

{'a1': 20, 'a2': 0}
-----------------------------------------------------------------------------


In [8]:
# Match 5
player_colors = {'black': 'a1', 'white': 'a2'}
a1_configs = {'player_color': 1, 'path': "utils_nov8/nov8/net37.model", 'debug': False}
a2_configs = {'player_color': 2, 'path': "utils_1/nov9-correct/net11.model", 'debug': False, 'play_random': True}
play_matches(a1_configs, a2_configs, player_colors=player_colors, tournament_no=5)


Starting Tournament #5
{'black': 'a1', 'white': 'a2'}


{'a1': 6, 'a2': 4}
{'a1': 13, 'a2': 7}

{'a1': 13, 'a2': 7}
-----------------------------------------------------------------------------


In [9]:
# Match 6
player_colors = {'black': 'a2', 'white': 'a1'}
a1_configs = {'player_color': 2, 'path': "utils_nov8/nov8/net37.model", 'debug': False}
a2_configs = {'player_color': 1, 'path': "utils_1/nov9-correct/net11.model", 'debug': False, 'play_random': True}
play_matches(a1_configs, a2_configs, player_colors=player_colors, tournament_no=6)


Starting Tournament #5
{'black': 'a2', 'white': 'a1'}


{'a1': 10, 'a2': 0}
{'a1': 19, 'a2': 1}

{'a1': 19, 'a2': 1}
-----------------------------------------------------------------------------


In [10]:
# Match 7
player_colors = {'black': 'a1', 'white': 'a2'}
a1_configs = {'player_color': 1, 'path': "utils_nov8/nov8/net37.model", 'debug': False, 'play_random': True}
a2_configs = {'player_color': 2, 'path': "utils_1/nov9-correct/net11.model", 'debug': False}
play_matches(a1_configs, a2_configs, player_colors=player_colors, tournament_no=5)


Starting Tournament #5
{'black': 'a1', 'white': 'a2'}


{'a1': 10, 'a2': 0}
{'a1': 20, 'a2': 0}

{'a1': 20, 'a2': 0}
-----------------------------------------------------------------------------


In [11]:
# Match 7
player_colors = {'black': 'a2', 'white': 'a1'}
a1_configs = {'player_color': 2, 'path': "utils_nov8/nov8/net37.model", 'debug': False, 'play_random': True}
a2_configs = {'player_color': 1, 'path': "utils_1/nov9-correct/net11.model", 'debug': False}
play_matches(a1_configs, a2_configs, player_colors=player_colors, tournament_no=5)


Starting Tournament #5
{'black': 'a2', 'white': 'a1'}


{'a1': 7, 'a2': 3}
{'a1': 13, 'a2': 7}

{'a1': 13, 'a2': 7}
-----------------------------------------------------------------------------


In [16]:
player_colors = {'black': 'a1', 'white': 'a2'}
a1_configs = {'player_color': 1, 'path': "utils_nov8/nov8/net55.model", 'debug': False}
a2_configs = {'player_color': 2, 'path': "utils_nov8/nov10/net1.model", 'debug': False}
play_matches(a1_configs, a2_configs, player_colors=player_colors, tournament_no=1)


Starting Tournament #1
{'black': 'a1', 'white': 'a2'}


{'a1': 10, 'a2': 0}
{'a1': 20, 'a2': 0}

{'a1': 20, 'a2': 0}
-----------------------------------------------------------------------------


In [15]:
player_colors = {'black': 'a2', 'white': 'a1'}
a1_configs = {'player_color': 2, 'path': "utils_nov8/nov8/net55.model", 'debug': False}
a2_configs = {'player_color': 1, 'path': "utils_nov8/nov10/net1.model", 'debug': False}
play_matches(a1_configs, a2_configs, player_colors=player_colors, tournament_no=1)


Starting Tournament #1
{'black': 'a2', 'white': 'a1'}


{'a1': 0, 'a2': 10}
{'a1': 0, 'a2': 20}

{'a1': 0, 'a2': 20}
-----------------------------------------------------------------------------


In [ ]:
player_colors = {'black': 'a2', 'white': 'a1'}
a1_configs = {'player_color': 2, 'path': "utils_nov8/nov8/net55.model", 'debug': False, 'p'}
a2_configs = {'player_color': 1, 'path': "utils_nov8/nov10/net1.model", 'debug': False}
play_matches(a1_configs, a2_configs, player_colors=player_colors, tournament_no=1)
